In [ ]:
import numpy as np
import itertools
import pandas as pd
from jax import grad, jit, vmap
import jax.numpy as jnp
import jax.scipy as jscipy
import matplotlib.pyplot as plt

In [ ]:
#Create combinations

count = 0
NumLevels = 10
MaxDiff = 7
NumRep = 1
combinations = []

for i in range(1, NumLevels - 2):
    for j in range(i + 1, NumLevels - 1):
        for k in range(j + 1, NumLevels):
            for l in range(k + 1, NumLevels + 1):
                count += 1
                combinations.append([i, j, k, l])


combinations = np.asarray(combinations)

combinations = combinations[np.abs(combinations[:, 3] - combinations[:, 2] - combinations[:, 1] + combinations[:, 0]) <= MaxDiff, :]


num_rep = NumRep
combinations = np.tile(combinations, (num_rep, 1))


In [ ]:
#Read the answer file
file_name = 'answers.csv'
BBDD = pd.read_csv(file_name)

In [ ]:
#Choose the distortion and the image
im_ref=1  #Reference Image (1,...,24)
transf = ['gaussian_noise'] #Distortion  (['rotation'] ['translation'] ['scale'] ['gaussian_noise'])


Nimages=10
BBDD = BBDD[BBDD['image_id_01'] == im_ref]
BBDD = BBDD[BBDD['transformation'].isin(transf)]

exps = BBDD[['distortion_level_11','distortion_level_12','distortion_level_21','distortion_level_22','answer']].values

exps = exps-1
exps = exps.astype('int')
exps = exps[exps[:,4]>-1]

In [ ]:
Np = 4000
psi = np.zeros((Nimages,))
psi = np.linspace(0,1,Nimages)
sig = 0.1

In [ ]:
def LL(x,psi,sig):
    LL = 0
    for n in range(x.shape[0]):
        #print(n)
        D =jnp.abs(psi[x[n,0]]-psi[x[n,1]])-jnp.abs(psi[x[n,2]]-psi[x[n,3]])
        #print(D)
        # Probabilidad de la elección
        pp = jscipy.stats.norm.cdf(D/sig)
        #print(pp)
        # Esto es para definir si es la probabilidad de elegir la de 0 (p) o la de 1 (1-p)
        aa = (x[n,4]-pp)
        bb = ((-1)**(1+x[n,4]))
        #print(aa,bb)
        LL = LL + jnp.log(aa*bb)
        #print(LL)
    return LL

In [ ]:
D_loss_D_psi = grad(LL,1)
D_loss_D_sig = grad(LL,2)

D_loss_D_psi = jit(D_loss_D_psi)
D_loss_D_sig = jit(D_loss_D_sig)

lr = 0.00001

BS = 20
cada = 100

LLs = np.zeros((Np,))
for ii in range(Np):

        iii = np.random.permutation(exps.shape[0])[0:BS]
        DD = D_loss_D_psi(exps[iii,:],psi,sig)
        DDs = D_loss_D_sig(exps[iii,:],psi,sig)

        psi[1:-1] =  psi[1:-1] + lr*DD[1:-1]
        sig = sig + lr*DDs

        LLs[ii] = LL(exps,psi,sig)
        if np.mod(ii,cada)==0:
            print(ii,np.round(LLs[ii],5),np.round(psi,3),np.round(sig,3))

        if np.isnan(LLs[ii]):
            break

In [ ]:
plt.figure()
SIGS = sig*np.ones(*psi.shape)
xxx = np.linspace(0,psi.shape[0]-1,psi.shape[0])
plt.errorbar(xxx,psi,SIGS)